In [1]:
import matplotlib.pyplot as plt
from joblib import load
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import sounddevice as sd
from features_functions import compute_features
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import classification_report

In [4]:


# Charger les données de sinus depuis le fichier pickle
f = open('bb1', 'rb')
bb_data = pickle.load(f)

print(len(bb_data))

50


In [5]:
%matplotlib notebook

In [7]:
with open('sinus1', 'rb') as f1:
    sin_data = pickle.load(f1)

print(len(sin_data))

50


In [8]:
sd.play(bb_data[0], 44100)

In [9]:
# Loop over the signals
learningFeatures = []
for k in sin_data:
    # Compute the signal in three domains
    sig_sq = k**2
    sig_t = k / np.sqrt(sig_sq.sum())
    sig_f = np.absolute(np.fft.fft(sig_t))
    sig_c = np.absolute(np.fft.fft(sig_f))

    # Compute the features and store them
    features_list = []
    N_feat, features_list = compute_features(sig_t, sig_f[:sig_t.shape[0]//2], sig_c[:sig_t.shape[0]//2])
    features_vector = np.array(features_list)[np.newaxis,:]

    # Store the obtained features in a np.arrays
    learningFeatures.append(features_vector)
    
for i in bb_data:
    # Compute the signal in three domains
    sig_sq = i**2
    sig_t = i / np.sqrt(sig_sq.sum())
    sig_f = np.absolute(np.fft.fft(sig_t))
    sig_c = np.absolute(np.fft.fft(sig_f))

    # Compute the features and store them
    features_list = []
    N_feat, features_list = compute_features(sig_t, sig_f[:sig_t.shape[0]//2], sig_c[:sig_t.shape[0]//2])
    features_vector = np.array(features_list)[np.newaxis,:]

    # Store the obtained features in a np.arrays
    learningFeatures.append(features_vector)
    
# Store the labels
v = np.zeros(50)
c = np.ones(50)
learningLabels = np.concatenate((v, c))

# Convert the learningFeatures list to a numpy array
learningFeatures = np.asarray(learningFeatures).reshape(100, 71)

# Encode the class names
# labelEncoder = preprocessing.LabelEncoder().fit(learningLabels)
# learningLabelsStd = labelEncoder.transform(learningLabels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(learningFeatures, learningLabels, test_size=0.2, random_state=42)
print(len(X_train),len(X_test),len(y_train),len(y_test))
# Learn the model
model = svm.SVC(C=10, kernel='linear', class_weight=None, probability=False)
scaler = preprocessing.StandardScaler(with_mean=True).fit(np.squeeze(X_train))
learningFeatures_scaled = scaler.transform(np.squeeze(X_train))
print(len(learningFeatures_scaled))
model.fit(learningFeatures_scaled, y_train)


# Scale the testing features using the scaler trained on the training data
X_test_scaled = scaler.transform(X_test)

# Predict the labels for the testing set
y_pred = model.predict(X_test_scaled)

# Print the classification report
print(classification_report(y_test, y_pred))

# Export the scaler and model on disk
# dump(scaler, "SCALER")
# dump(model, "SVM_MODEL")


80 20 80 20
80
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        12
         1.0       1.00      1.00      1.00         8

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



In [ ]:
y_test